## Dependencias 

In [14]:
import numpy as np
import pandas as pd 

from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR,CHAR,INTEGER,BIGINT,BOOLEAN,DATE,DATETIME,FLOAT

import json
pd.set_option('display.max_columns',None)

## Lectura de datos 

In [15]:
ruta = '/media/jose/090f6b94-de30-4aaf-9f8a-4e18b120d7f6/bd/02.  Para Ingeniería/cc_txn/credit_card_transactions-ibm_v2.csv'

In [16]:
data = pd.read_csv(ruta,dtype=str)
data.shape

(24386900, 15)

In [19]:
data = data.sample(100000).reset_index(drop=True)

In [20]:
data.head(2)

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,1986,0,2016,11,26,12:49,$39.46,Chip Transaction,2910328604019547969,Knoxville,TN,37918.0,7538,NaN,No
1,1075,3,2008,9,16,15:20,$89.94,Swipe Transaction,4449607287528943014,Clinton,TN,37716.0,5411,NaN,No


## Conexión a BD

In [21]:
creds = json.load(open('../../creds/mysql.json','r'))

In [22]:
cnx = create_engine(f"mysql+pymysql://{creds['u']}:{creds['p']}@{creds['h']}/{creds['d']}").connect()

## Entidades

### Comercio

In [23]:
entComercio = data[['Merchant Name']].drop_duplicates().reset_index(drop=True)
entComercio.insert(0,'id',entComercio.index+1)
entComercio.columns = ['id','merchant_name']
entComercio['merchant_name'] = entComercio['merchant_name'].map(str)
entComercio.head()

,id,merchant_name
0,1,2910328604019547969
1,2,4449607287528943014
2,3,2367739630280360626
3,4,-2088492411650162548
4,5,-6160036380778658394


In [24]:
entComercio['merchant_name'].map(len).max()

20

In [25]:
dtypes = [INTEGER,VARCHAR(20)]
entComercio.to_sql(name='tbl_merchant',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,
               dtype=dict(zip(entComercio.columns,dtypes)))

11362

### Territorio

In [29]:
entTerritorio = pd.read_clipboard()

In [30]:
entTerritorio.tail()

,id,territory_code,territory_name
220,221,170,Paraguay
221,222,171,Kiribati
222,223,172,Botswana
223,224,173,Online
224,225,AA,Diplomatic Post Office


In [31]:
dtypes = [INTEGER,VARCHAR(3),VARCHAR(32)]
entTerritorio.to_sql(name='tbl_territory',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,
               dtype=dict(zip(entTerritorio.columns,dtypes)))

225

### Ciudad

In [32]:
data['Merchant State'] = np.where(data['Merchant City']=='ONLINE','Online',data['Merchant State'])

In [33]:
entCiudad = data[['Merchant City','Merchant State']].drop_duplicates()

In [34]:
entCiudad.columns = ['territory_city','territory_name']

In [35]:
aux = entCiudad.merge(entTerritorio,how='left',on=['territory_name'])
aux['id'] =np.where(aux['id']<52,None,aux['id']) 
aux = aux.merge(entTerritorio,how='left',
                left_on=['territory_name'],right_on=['territory_code'])
aux.tail()


,territory_city,territory_name_x,id_x,territory_code_x,id_y,territory_code_y,territory_name_y
6539,Hazen,AR,NaN,NaN,29.0,AR,Arkansas
6540,Winfield,KS,NaN,NaN,32.0,KS,Kansas
6541,Beckemeyer,IL,NaN,NaN,6.0,IL,Illinois
6542,Sergeant Bluff,IA,NaN,NaN,28.0,IA,Iowa
6543,Jamestown,PA,NaN,NaN,7.0,PA,Pennsylvania


In [36]:
aux['id_territory'] = (np.where(aux['id_y'].isnull(),aux['id_x'],aux['id_y'])).astype(int)

In [37]:
aux = aux[['id_territory','territory_city']].dropna().reset_index(drop=True)
aux.insert(0,'id',aux.index+1)

In [38]:
aux['territory_city'].map(len).max()

26

In [39]:
dtypes = [INTEGER,INTEGER,VARCHAR(26)]
aux.to_sql(name='tbl_city',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,
               dtype=dict(zip(aux.columns,dtypes)))

6544

### Tipo de Transacción


In [40]:
entTipoTxn = data[['Use Chip']].drop_duplicates().reset_index(drop=True)
entTipoTxn.insert(0,'id',entTipoTxn.index+1)
entTipoTxn.columns = ['id','txn_type']
entTipoTxn['txn_type'].map(len).max()

18

In [41]:
dtypes = [INTEGER,VARCHAR(18)]
entTipoTxn.to_sql(name='tbl_txn_type',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entTipoTxn.columns,dtypes)))

3

### Código Postal

In [42]:
entZip = data[['Merchant City','Merchant State','Zip']].drop_duplicates().dropna().reset_index(drop=True)
entZip['Zip'] = entZip['Zip'].map(float).astype(int).map(lambda x: f'{x:05d}')
entZip = entZip.merge(entTerritorio,how='inner',left_on=['Merchant State'],right_on=['territory_code'])
entZip = entZip.merge(aux[['id','territory_city','id_territory']],how='inner',left_on=['Merchant City','id'],right_on=['territory_city','id_territory'])
entZip = entZip[['Zip','id_y']]
entZip.columns = ['zip_code','id_city']
entZip.insert(0,'id',(entZip.index+1).astype(int))
entZip.head()

,id,zip_code,id_city
0,1,37918,1
1,2,37931,1
2,3,37917,1
3,4,37938,1
4,5,37933,1


In [43]:
dtypes = [INTEGER,CHAR(5),INTEGER]
entZip.to_sql(name='tbl_zip',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entZip.columns,dtypes)))

9506

### MCC - Merchant Category Code

In [44]:
mcc = data[['MCC']].drop_duplicates().dropna().reset_index(drop=True).astype(int)

In [45]:
entMCC = pd.read_csv('https://raw.githubusercontent.com/greggles/mcc-codes/main/mcc_codes.csv')
entMCC['mcc'] = entMCC['mcc'].map(int)
entMCC= entMCC[['mcc','edited_description']]
entMCC.columns = ['mcc','mcc_description']
entMCC.insert(0,'id',entMCC.index+1)
entMCC.head()

,id,mcc,mcc_description
0,1,742,Veterinary Services
1,2,763,Agricultural Co-operatives
2,3,780,"Horticultural Services, Landscaping Services"
3,4,1520,General Contractors-Residential and Commercial
4,5,1711,Air Conditioning Contractors – Sales and Insta...


In [46]:
entMCC['mcc_description'].map(len).max()

142

In [47]:
dtypes = [INTEGER,INTEGER,VARCHAR(150)]
entMCC.to_sql(name='tbl_mcc',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entMCC.columns,dtypes)))

981

### User

In [48]:
entUser = data[['User']].drop_duplicates().reset_index(drop=True)
entUser.columns =['id']

dtypes = [INTEGER]
entUser.to_sql(name='tbl_user',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entUser.columns,dtypes)))

1755

### Card

In [49]:
entCard = data[['User','Card']].drop_duplicates().reset_index(drop=True)
entCard.columns = ['id_user','card_number']
entCard.insert(0,'id',entCard.index+1)
entCard.head()

,id,id_user,card_number
0,1,1986,0
1,2,1075,3
2,3,1110,0
3,4,975,0
4,5,414,0


In [50]:
dtypes = [INTEGER]*3
entCard.to_sql(name='tbl_card',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entCard.columns,dtypes)))

5019

### Errores 

In [52]:
data.insert(0,'id',data.index+1)

2

In [61]:
entError = data[['id','Errors?']].dropna().reset_index(drop=True)
maxErrors = entError['Errors?'].map(lambda x:x.split(',')).map(len).max()
errors = [f'error_{i+1}' for i in range(maxErrors)]
entError[errors] = entError['Errors?'].map(lambda x:x.split(',')).apply(pd.Series)
entError.head()
entError = entError.melt(id_vars=['id'],value_vars=errors).dropna().reset_index(drop=True).drop('variable',axis=1)
entError = entError[['value']].drop_duplicates().reset_index(drop=True) 

In [62]:
entError.columns = ['error_description']
entError.insert(0,'id',entError.index+1)
entError.head()

,id,error_description
0,1,Insufficient Balance
1,2,Technical Glitch
2,3,Bad PIN
3,4,Bad Expiration
4,5,Bad Card Number


In [63]:
dtypes = [INTEGER,VARCHAR(20)]
entError.to_sql(name='tbl_error',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entError.columns,dtypes)))

7

### Transacción 

In [64]:
data.head(2)

,id,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,1,1986,0,2016,11,26,12:49,$39.46,Chip Transaction,2910328604019547969,Knoxville,TN,37918.0,7538,NaN,No
1,2,1075,3,2008,9,16,15:20,$89.94,Swipe Transaction,4449607287528943014,Clinton,TN,37716.0,5411,NaN,No


In [65]:

entTxn = data.copy()
entTxn['Amount'] = entTxn['Amount'].map(lambda x: x.replace('$','')).map(float)
entTxn['Zip'] =entTxn['Zip'].fillna(99999.0).astype(float).astype(int)
entTxn['Zip'] = np.where(entTxn['Zip']==99999,None,entTxn['Zip'].map(lambda x: f'{x:05d}'))
entTxn['MCC'] = entTxn['MCC'].fillna(9999).astype(int)
entTxn['MCC'] = np.where(entTxn['MCC']==9999,None,entTxn['MCC'])

entTxn['Year'] = entTxn['Year'].astype(str)
entTxn['Month'] = entTxn['Month'].astype(int).map(lambda x: f'{x:02d}')
entTxn['Day'] = entTxn['Month'].astype(int).map(lambda x: f'{x:02d}')

entTxn['txn_date'] = entTxn[['Year','Month','Day']].apply('-'.join,axis=1)
entTxn['ts'] = entTxn[['txn_date','Time']].apply(' '.join,axis=1)

entTxn['ts'] = pd.to_datetime(entTxn['ts'],format='%Y-%m-%d %H:%M')
entTxn.drop(['Year','Month','Day','Time','txn_date','Errors?'],axis=1,inplace=True)

entTxn.rename(columns={'User':'id_user','Card':'card_number'},inplace=True) 
entTxn = entTxn.merge(entCard.rename(columns={'id':'id_card'}),how='left',on=['id_user','card_number'])
entTxn.drop('card_number',axis=1,inplace=True)

entTxn.rename(columns={'Use Chip':'txn_type'},inplace=True) 
entTxn = entTxn.merge(entTipoTxn.rename(columns={'id':'id_txn_type'}),how='left',on=['txn_type'])
entTxn.drop('txn_type',axis=1,inplace=True)

entTxn.rename(columns={'Merchant Name':'merchant_name'},inplace=True) 
entTxn = entTxn.merge(entComercio.rename(columns={'id':'id_comercio'}),how='left',on=['merchant_name'])
entTxn.drop('merchant_name',axis=1,inplace=True)

entTxn = entTxn.merge(entTerritorio.rename(columns={'id':'id_territory'}),
                      how='left',
                      left_on=['Merchant State'],right_on=['territory_code'],indicator=True)

entTxn['Merchant State'] = np.where(entTxn['_merge']=='left_only',entTxn['Merchant State'],None)
entTxn = entTxn.merge(entTerritorio.rename(columns={'id':'id_territory'}),
                      how='left',
                      left_on=['Merchant State'],right_on=['territory_name'])
entTxn['id_territory'] = (np.where(entTxn['id_territory_x'].isnull(),entTxn['id_territory_y'],entTxn['id_territory_x'])).astype(int)
entTxn.drop(['id_territory_x','territory_code_x','territory_name_x','_merge','id_territory_y','territory_code_y','territory_name_y'],axis=1,inplace=True)
entTxn.drop(['Merchant City','Merchant State'],axis= 1,inplace=True)   

entTxn.rename(columns={'Zip':'zip_code'},inplace=True) 
entTxn = entTxn.merge(entZip.rename(columns={'id':'id_zip'}),
                      how='left',
                      on='zip_code')
entTxn.drop('zip_code',axis=1,inplace=True)

entTxn.rename(columns={'MCC':'mcc'},inplace=True) 
entTxn = entTxn.merge(entMCC.rename(columns={'id':'id_mcc'}).drop('mcc_description',axis=1),
                      how='left',
                      on='mcc')
entTxn.drop('mcc',axis=1,inplace=True)

entTxn.rename(columns={'Is Fraud?' : 'is_fraud'},inplace=True)
entTxn['is_fraud'] = entTxn['is_fraud']=='Yes'

entTxn.rename(columns={'Amount':'amount'},inplace=True)

In [67]:
dtypes = [INTEGER]*2+[FLOAT,BOOLEAN,DATETIME]+[INTEGER]*7
entTxn.to_sql(name='tbl_txn',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entTxn.columns,dtypes)))

100000

### Transacción-Error

In [68]:
entErrTxn = data[['id','Errors?']].dropna().reset_index(drop=True)
errors = [f'error_{i+1}' for i in range(maxErrors)]
entErrTxn[errors] = entErrTxn['Errors?'].map(lambda x:x.split(',')).apply(pd.Series)
entErrTxn = entErrTxn.melt(id_vars=['id'],value_vars=errors).dropna().reset_index(drop=True).drop('variable',axis=1)
entErrTxn = entErrTxn.merge(entError.rename(columns={'id':'id_error'}),left_on='value',right_on='error_description').drop(['value','error_description'],axis=1).rename(columns={'id':'id_txn'})

In [69]:
dtypes = [INTEGER]*2
entErrTxn.to_sql(name='tbl_txn_error',
               index=False,
               con=cnx,
               if_exists='replace',
               chunksize=5000,

               dtype=dict(zip(entErrTxn.columns,dtypes)))

1618